In [1]:
import os
import pandas as pd
from collections import defaultdict
from functools import partial
from multiprocessing import cpu_count
from pathlib import Path
from textwrap import dedent
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
import torch as tf
from collections import Counter

In [2]:
def compute_label(input_front, input_back, convert_dict):
    if input_front['write'] > 0.9:
        out_label = convert_dict['write']
    elif input_front['talk'] > 0.9:
        out_label = convert_dict['talk']
    elif input_front['read'] > 0.5:
        out_label = convert_dict['read']
    elif input_front['drink'] > 0.9:
        out_label = convert_dict['drink']
    elif input_front['eat'] > 0.4:
        out_label = convert_dict['eat']
    elif input_back['computer'] > 0.6:
        out_label = convert_dict['computer']
    else:
        out_label = 6
    return out_label

In [3]:
def load_sensor_data_without_h(fname):
    sensor_txt = np.genfromtxt(fname, delimiter=',', dtype=None, encoding=None)
    # a 2-4 w(augular velocity) 6-8 Angle 10-12 h 14-16 q(quaternion) 18-21
    # a 0-2 w 3-5 Angle 6-8 q 9 10 11 12
    row_len = 3*3 + 4
    data_length = len(sensor_txt)
    sensor_array = np.zeros((data_length, row_len))
    for row_i, sensor_row in enumerate(sensor_txt):
        # a 2-4
        sensor_array[row_i, 0] = sensor_row[2]
        sensor_array[row_i, 1] = sensor_row[3]
        sensor_array[row_i, 2] = sensor_row[4]
        # w 6-8
        sensor_array[row_i, 3] = sensor_row[6]
        sensor_array[row_i, 4] = sensor_row[7]
        sensor_array[row_i, 5] = sensor_row[8]
        # Angle 10-12
        sensor_array[row_i, 6] = sensor_row[10]
        sensor_array[row_i, 7] = sensor_row[11]
        sensor_array[row_i, 8] = sensor_row[12]
        # q 18-21
        sensor_array[row_i, 9] = sensor_row[18]
        sensor_array[row_i, 10] = sensor_row[19]
        sensor_array[row_i, 11] = sensor_row[20]
        sensor_array[row_i, 12] = sensor_row[21]
    return sensor_array
def sample_sensor_data(input_data, window_sz = 128, sample_sz = 128):
    sensor_length = input_data.shape[0]
    print('The shape of sensor input data', input_data.shape)
    feature_sz = input_data.shape[1]
    data_sz = 0
    print('the length of sensor', sensor_length)
    for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        data_sz = data_sz + 1
    all_sensor_data = np.zeros((data_sz, feature_sz, window_sz))
    cnt = 0
    for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        sample = input_data[i:i + window_sz, :]
        sample = np.transpose(sample)
        all_sensor_data[cnt, :, :] = sample
        cnt = cnt + 1
    print('the shape of sensor dataset', all_sensor_data.shape)
    return all_sensor_data 

In [4]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.

    The separable convlution is a method to reduce number of the parameters
    in the deep learning network for slight decrease in predictions quality.
    """

    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.

    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """

    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):

        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)


class Classifier_dh(nn.Module):
    def __init__(self, raw_ni, raw_sz, no, drop=0.05):
        super().__init__()
        self.conv1 = SepConv1d(raw_ni, 32, 8, 2, 3, drop=drop)
        self.conv2 = SepConv1d(32, 64, 8, 4, 2, drop=drop)
        self.conv3 = SepConv1d(64, 128, 8, 4, 2, drop=drop)
        self.conv4 = SepConv1d(128, 256, 8, 4, 2)
        self.flatten = Flatten()
        self.fc1 = nn.Sequential(nn.Dropout(drop), nn.Linear(raw_sz*2, 64), nn.ReLU(inplace=True))
        self.fc2 = nn.Sequential(nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True))
        self.out = nn.Sequential(
            nn.Linear(64, 64), nn.ReLU(inplace=True), nn.Linear(64, no))

    def forward(self, t_raw):
#         print('input size', t_raw.size())
        x = self.conv1(t_raw)
#         print('conv1', x.size())
        x = self.conv2(x)
#         print('conv2', x.size())
        x = self.conv3(x)
#         print('conv3', x.size())
        x = self.conv4(x)
#         print('conv4', x.size())
        x = self.flatten(x)
#         print('flatten', x.size())
        x = self.fc1(x)
#         print('fc1', x.size())
        x = self.fc2(x)
#         print('fc2', x.size())
        x = self.out(x)
        return x

In [5]:
# Program to find most frequent  
# element in a list 
from collections import Counter 
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 

In [6]:
dirpath = "C:\\Users\\zhjsc\\Desktop\\zongyuan\\sensor\\repos\\time_series_network\\sensor_data_v6\\"
frontpath = dirpath + "front_angle_7action.csv"
front_csv = pd.read_csv(frontpath)
backpath = dirpath + "back_angle_7action.csv"
back_csv = pd.read_csv(backpath)

In [7]:
front_shape = front_csv.shape
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
label_array = []
for i in range(front_shape[0]):
    front_row = front_csv.iloc[i]
    back_row = back_csv.iloc[i]
    label = compute_label(input_front=front_row, input_back=back_row, convert_dict=txt_to_label)
    label_array.append(label)
print(len(label_array))

42990


In [8]:
head_sensor = load_sensor_data_without_h(fname='sensor_data_v6\WT01—head.txt')
head_sensor_data = sample_sensor_data(head_sensor)

The shape of sensor input data (74298, 13)
the length of sensor 74298
the shape of sensor dataset (579, 13, 128)


In [9]:
# Generate labels for the training dataset
sensor_length = head_sensor.shape[0]
window_sz = 128
sample_sz = 128
# label = pd.read_csv('video_20200917.csv', index_col=0)
label_array = np.asarray(label_array)
def sensor_to_slowfast(sensor_index, sensor_data, label_data):
    slowfast_index = int(sensor_index  / sensor_data.shape[0] * label_data.shape[0])
    return slowfast_index
all_label = np.zeros((head_sensor_data.shape[0], 1)) 
cnt = 0

for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
    start = sensor_to_slowfast(sensor_index=i, sensor_data = head_sensor, label_data=label_array)
    end = sensor_to_slowfast(sensor_index=i+window_sz, sensor_data = head_sensor, label_data=label_array)
    cur_label_array = label_array[start:end]
    all_label[cnt] = most_frequent(cur_label_array)
    cnt = cnt + 1
bs = 4
sz = all_label.shape[0]
trn_ds = TensorDataset(torch.tensor(head_sensor_data).float(), torch.tensor(all_label).long())
trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=0)

In [10]:
# build the validation dataset
val_head_sensor = load_sensor_data_without_h(fname='sensor_data_v3\head_20200917.txt')
val_head_sensor_data = sample_sensor_data(val_head_sensor)

The shape of sensor input data (14682, 13)
the length of sensor 14682
the shape of sensor dataset (113, 13, 128)


In [11]:
# path of csv file 
dirpath = "C:\\Users\\zhjsc\\Desktop\\zongyuan\\sensor\\repos\\time_series_network\\sensor_data_v3\\"
frontpath = dirpath + "front_angle_7action.csv"
# load csv using pandas 
val_front_csv = pd.read_csv(frontpath)
val_front_shape = val_front_csv.shape
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
val_label_slowfast = []

for i in range(val_front_shape[0]):
    front_row = val_front_csv.iloc[i]
    back_row = front_row.copy()
    label = compute_label(input_front=front_row, input_back=back_row, convert_dict=txt_to_label)
    val_label_slowfast.append(label)

# convert slowfast label to sensor label 
from collections import Counter
val_sensor_length = val_head_sensor.shape[0]
val_label = np.zeros((val_head_sensor_data.shape[0], 1))
cnt = 0
for i in range(0, val_sensor_length-window_sz-sample_sz, sample_sz):
    start = sensor_to_slowfast(sensor_index=i, sensor_data = head_sensor, label_data=label_array)
    end = sensor_to_slowfast(sensor_index=i+window_sz, sensor_data = head_sensor, label_data=label_array)
    cur_label_array = val_label_slowfast[start:end]
    val_label[cnt] = most_frequent(cur_label_array)
    cnt = cnt + 1

In [12]:
val_label = np.asarray(val_label)
val_ds = TensorDataset(torch.tensor(val_head_sensor_data).float(), torch.tensor(val_label).long())
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)
print(val_head_sensor_data.shape)
print(val_label.shape)

(113, 13, 128)
(113, 1)


In [13]:
lr = 0.001
n_epochs = 300
iterations_per_epoch = len(trn_ds)
num_classes = 7
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Classifier_dh(head_sensor_data.shape[1], head_sensor_data.shape[2], num_classes).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)
print('Start model training')

for epoch in range(1, n_epochs + 1):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        x_raw, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        out = model(x_raw)
        y_batch = tf.squeeze(y_batch)
#         print(out.size(), y_batch.size())
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()

    loss_history.append(epoch_loss)
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        y_batch = tf.squeeze(y_batch)
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        if preds.size()[0] > 1:
            total += y_batch.size(0)
            correct += (preds == y_batch).sum().item()

    acc = correct / total
    acc_history.append(acc)
    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch:   1. Loss: 1006.3194. Acc.: 48.21%
Epoch 1 best model saved with accuracy: 48.21%
Epoch:   2. Loss: 949.9478. Acc.: 48.21%
Epoch:   4. Loss: 772.9881. Acc.: 16.96%
Epoch:   8. Loss: 651.5449. Acc.: 6.25%
Epoch:  16. Loss: 537.5355. Acc.: 8.93%
Epoch:  32. Loss: 324.9703. Acc.: 1.79%
Epoch:  64. Loss: 99.0955. Acc.: 1.79%
Epoch: 128. Loss: 66.0448. Acc.: 1.79%
Epoch: 256. Loss: 29.7042. Acc.: 1.79%
